This code was used to quantify the number of lipid units (including single and clumped lipid droplets) per cell after knock down of
TAX1BP1 or ADAMTS19 versus scrambled control, in the presence and absence of endogenous αSyn.

Import Libraries

In [2]:
import czifile
import numpy as np
from scipy.ndimage import zoom
import time
from cellpose import models, io, plot
import czifile
import numpy as np
from skimage.filters import gaussian, threshold_otsu
from skimage.measure import label, regionprops
import matplotlib.pyplot as plt
import cv2
from IPython.display import Image, display
from skimage.color import label2rgb, rgb2hsv
from skimage.filters import gaussian, sobel
from skimage.measure import label, regionprops
from skimage.morphology import binary_erosion, binary_dilation, disk, local_maxima
from scipy.ndimage import binary_fill_holes
from skimage.feature import blob_log
from skimage.color import rgb2gray
from skimage.draw import circle_perimeter
from matplotlib.colors import LogNorm
import scipy.ndimage as ndi
import pandas as pd
import os
from skimage import morphology, io
from IPython.display import clear_output
from scipy.ndimage import zoom
import time
import matplotlib.pyplot as plt
model = models.Cellpose(model_type='nuclei')

Define Sub Functions

In [3]:
def calculate_circularity(area, perimeter):
    """
    Calculates the circularity of a region.

    Args:
        area: The area of the region.
        perimeter: The perimeter of the region.

    Returns:
        Circularity value (a measure of how circular a region is).
    """
    circularity = (4 * np.pi * area) / (perimeter**2)
    return circularity


def no_yfp_images(image_path, basename):
    """
    Analyzes an image without YFP channel, identifying nuclei and lipid droplets.

    Args:
        image_path: The path to the .czi image file.
        basename: The base name for the output DataFrame.

    Returns:
        lipid_df: DataFrame containing information about detected lipid droplets.
        count_df: DataFrame containing summary counts of detected features.
    """
    # Load and preprocess the image
    image = czifile.imread(image_path)
    image_squeezed = np.squeeze(image)
    red_channel = image_squeezed[0, :, :]
    blue_channel = image_squeezed[1, :, :]
    
    # Detect nuclei using the blue channel
    masks, flows, styles, diams = model.eval(blue_channel, diameter=150, channels=[0, 0])
    labeled_nuclei = label(masks)
    nuclei = [nuclei.area for nuclei in regionprops(labeled_nuclei)]

    # Thresholding the red channel
    threshold = threshold_otsu(gaussian(red_channel))
    red_channel = gaussian(red_channel) > threshold
    labeled_droplets = label(red_channel)

    lipid_sizes = []
    lipid_circularity = []
    lipid_label = []

    # Analyze lipid droplets
    for region in regionprops(labeled_droplets):
        lipid_size = region.area
        circularity = calculate_circularity(lipid_size, region.perimeter)
        if str(circularity) == 'inf':
            continue
        
        lipid_sizes.append(lipid_size)
        lipid_circularity.append(circularity)
        lipid_label.append('single' if circularity > 0.65 else 'bunched') # Circularity cutoff to differentiate between single and bunched lipids

    lipid_df = pd.DataFrame({
        "Filename": [basename] * len(lipid_sizes),
        "Lipid_Size": lipid_sizes,
        "Lipid_Circularity": lipid_circularity,
        "Lipid_Label": lipid_label,
        "Lipid_Syn_Association": ['unknown'] * len(lipid_sizes)
    })

    count_single = lipid_label.count('single')
    count_bunched = lipid_label.count('bunched')

    count_df = pd.DataFrame({
        "Filename": [basename],
        "Nuclei": len(nuclei),
        "Single_Lipids": count_single,
        "Bunched_Lipids": count_bunched,
        "Total_Lipids": len(lipid_sizes),
        "Synuclein_Associated_Lipids": 0  # Placeholder value
    })

    return lipid_df, count_df


def yfp_images(image_path, basename):
    """
    Analyzes an image with YFP channel, identifying nuclei, lipid droplets, and synuclein association.

    Args:
        image_path: The path to the .czi image file.
        basename: The base name for the output DataFrame.

    Returns:
        lipid_df: DataFrame containing information about detected lipid droplets.
        count_df: DataFrame containing summary counts of detected features.
    """
    # Load and preprocess the image
    image = czifile.imread(image_path)
    image_squeezed = np.squeeze(image)
    green_channel = image_squeezed[1, :, :]
    green_channel = gaussian(green_channel)
    red_channel = image_squeezed[0, :, :]
    blue_channel = image_squeezed[2, :, :]
    
    # Detect nuclei using the blue channel
    masks, flows, styles, diams = model.eval(blue_channel, diameter=150, channels=[0, 0])
    labeled_nuclei = label(masks)
    nuclei = [nuclei.area for nuclei in regionprops(labeled_nuclei)]

    # Thresholding the red channel
    threshold = threshold_otsu(gaussian(red_channel))
    red_channel = gaussian(red_channel) > threshold
    labeled_droplets = label(red_channel)

    lipid_sizes = []
    lipid_circularity = []
    lipid_label = []
    lipid_syn_association = []

    # Analyze lipid droplets
    for region in regionprops(labeled_droplets):
        lipid_size = region.area
        circularity = calculate_circularity(lipid_size, region.perimeter)
        if str(circularity) == 'inf':
            continue

        lipid_sizes.append(lipid_size)
        lipid_circularity.append(circularity)
        lipid_label.append('single' if circularity > 0.65 else 'bunched') # Circularity cutoff to differentiate between single and bunched lipids

        lipid_mask = labeled_droplets == region.label
        
        # Check for synuclein association
        syn_under_lipids = np.mean(green_channel[lipid_mask > 0])
        lipid_syn_association.append('yes' if syn_under_lipids > 0.5 else 'no')
    
    lipid_df = pd.DataFrame({
        "Filename": [basename] * len(lipid_sizes),
        "Lipid_Size": lipid_sizes,
        "Lipid_Circularity": lipid_circularity,
        "Lipid_Label": lipid_label,
        "Lipid_Syn_Association": lipid_syn_association
    })

    count_single = lipid_label.count('single')
    count_bunched = lipid_label.count('bunched')
    count_syn_association = lipid_syn_association.count('yes')

    count_df = pd.DataFrame({
        "Filename": [basename],
        "Nuclei": len(nuclei),
        "Single_Lipids": count_single,
        "Bunched_Lipids": count_bunched,
        "Total_Lipids": len(lipid_sizes),
        "Synuclein_Associated_Lipids": count_syn_association
    })

    return lipid_df, count_df


Define Main Function

In [4]:
def main(folder_path):
    lipid_all_data = []
    all_data = []

    # Iterate through each image in the specified folder
    for well_image in os.listdir(folder_path):
        if well_image.lower().endswith(".czi"):  # Filter for CZI files
            well_image_path = os.path.join(folder_path, well_image)
            well_image_base_name = os.path.basename(well_image)[:-4]

            # Determine if the image has a YFP channel and analyze accordingly
            if "no" in well_image_base_name.lower():  # Signals that the image does not have a YFP channel
                lipid_df, count_df = no_yfp_images(well_image_path, well_image_base_name)
            else:
                lipid_df, count_df = yfp_images(well_image_path, well_image_base_name)
            
            # Append the results to the respective lists
            lipid_all_data.append(lipid_df)
            all_data.append(count_df)
            
            # Clear the output to keep the console clean
            clear_output(wait=True)

    # Combine the data from all images into a single DataFrame
    combined_df = pd.concat(all_data, ignore_index=True)
    combined_lipid_df = pd.concat(lipid_all_data, ignore_index=True)

    # Save each DataFrame to its own Excel file
    combined_df.to_excel("COUNT.xlsx", index=False)
    combined_lipid_df.to_excel("LIPIDS.xlsx", index=False)

# Example usage:
if __name__ == "__main__":
    folder_path = r"test_images"
    main(folder_path)


21
